# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")
city_data_df = pd.DataFrame(city_data_df)
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,296.59,92,100,8.75,US,1731303194
1,1,port-aux-francais,-49.3500,70.2167,278.21,87,74,8.54,TF,1731303304
2,2,grytviken,-54.2811,-36.5092,271.89,88,100,1.07,GS,1731303305
3,3,gadzhiyevo,69.2551,33.3362,276.48,73,100,7.39,RU,1731303306
4,4,edinburgh of the seven seas,-37.0676,-12.3116,289.33,90,93,16.46,SH,1731303307


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
city_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,296.59,92,100,8.75,US,1731303194
1,1,port-aux-francais,-49.3500,70.2167,278.21,87,74,8.54,TF,1731303304
2,2,grytviken,-54.2811,-36.5092,271.89,88,100,1.07,GS,1731303305
3,3,gadzhiyevo,69.2551,33.3362,276.48,73,100,7.39,RU,1731303306
4,4,edinburgh of the seven seas,-37.0676,-12.3116,289.33,90,93,16.46,SH,1731303307


In [5]:
#Convert Max Temp from Kelvin to Celsius
city_data_df['Max Temp'] = pd.to_numeric(city_data_df['Max Temp'], errors='coerce')
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,23.44,92,100,8.75,US,1731303194
1,1,port-aux-francais,-49.3500,70.2167,5.06,87,74,8.54,TF,1731303304
2,2,grytviken,-54.2811,-36.5092,-1.26,88,100,1.07,GS,1731303305
3,3,gadzhiyevo,69.2551,33.3362,3.33,73,100,7.39,RU,1731303306
4,4,edinburgh of the seven seas,-37.0676,-12.3116,16.18,90,93,16.46,SH,1731303307
...,...,...,...,...,...,...,...,...,...,...
586,586,morant bay,17.8815,-76.4093,25.78,84,51,1.66,JM,1731304011
587,587,la foa,-21.7108,165.8276,25.51,68,63,4.64,NC,1731304012
588,588,chela,34.0690,104.4619,6.93,77,100,0.61,CN,1731304014
589,589,ghat,24.9647,10.1728,16.88,32,25,2.63,LY,1731304015


In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_filtered = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]
city_data_df_filtered.head()
# Drop any rows with null values
city_data_df_filtered = city_data_df_filtered.dropna()

# Display sample data
city_data_df_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,macapa,0.0389,-51.0664,26.99,89,0,2.57,BR,1731303379
99,99,chinde,26.8212,87.7095,26.08,49,0,2.44,NP,1731303414
335,335,baiti,13.7450,-14.7482,24.08,25,0,3.37,SN,1731303726
348,348,tazacorte,28.6290,-17.9293,25.64,30,0,4.02,ES,1731303741
396,396,mermoz boabab,14.7065,-17.4758,24.88,83,0,4.12,SN,1731303797
500,500,umuarama,-23.7664,-53.3250,21.24,76,0,3.67,BR,1731303886


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filtered.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
69,macapa,BR,0.0389,-51.0664,89,
99,chinde,NP,26.8212,87.7095,49,
335,baiti,SN,13.7450,-14.7482,25,
348,tazacorte,ES,28.6290,-17.9293,30,
396,mermoz boabab,SN,14.7065,-17.4758,83,
500,umuarama,BR,-23.7664,-53.3250,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
limit = 20

params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
macapa - nearest hotel: Equatorial Hotel
chinde - nearest hotel: No hotel found
baiti - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
mermoz boabab - nearest hotel: La maison blanche
umuarama - nearest hotel: Ibis Styles


,City,Country,Lat,Lng,Humidity,Hotel Name
69,macapa,BR,0.0389,-51.0664,89,Equatorial Hotel
99,chinde,NP,26.8212,87.7095,49,No hotel found
335,baiti,SN,13.7450,-14.7482,25,No hotel found
348,tazacorte,ES,28.6290,-17.9293,30,App Leyma
396,mermoz boabab,SN,14.7065,-17.4758,83,La maison blanche
500,umuarama,BR,-23.7664,-53.3250,76,Ibis Styles


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)